In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%config IPCompleter.greedy=True

In [ ]:
# sns.set_style('dark')
sns.set()
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['lines.markeredgewidth'] = 1
# plt.style.use("dark_background")


In [ ]:
current_palette = sns.color_palette()
sns.palplot(current_palette)

In [ ]:
xl_file_electriciteit_productie_en_productiemiddelen = pd.ExcelFile('data/CBS/Elektriciteit__productie_en_productiemiddelen.xlsx')
df_elec_prod = xl_file_electriciteit_productie_en_productiemiddelen.parse(xl_file_electriciteit_productie_en_productiemiddelen.sheet_names[0]).fillna(0).set_index('Perioden')
xl_file_elektriciteit_balans = pd.ExcelFile('data/CBS/Elektriciteitsbalans__aanbod_en_verbruik.xlsx')
df_elec_balans = xl_file_elektriciteit_balans.parse(xl_file_elektriciteit_balans.sheet_names[0]).fillna(0)
xl_file_hernieuwbare_productie = pd.ExcelFile('data/CBS/Hernieuwbare_elektriciteit__productie_en_vermogen.xlsx')
df_hernieuwbare_productie = xl_file_hernieuwbare_productie.parse(xl_file_hernieuwbare_productie.sheet_names[0]).fillna(0)
xl_file_hernieuwbare_verbruik = pd.ExcelFile('data/CBS/Hernieuwbare_energie__verbruik_naar_energiebron.xlsx')
df_hernieuwbare_verbruik = xl_file_hernieuwbare_verbruik.parse(xl_file_hernieuwbare_verbruik.sheet_names[0]).fillna(0)

In [ ]:
df_elec_prod_totaal = df_elec_prod[df_elec_prod['type'] == 'Totaal'].drop('type', 1)
df_elec_balans = df_elec_balans.T.rename(columns=df_elec_balans.T.iloc[0])
df_elec_balans = df_elec_balans.drop(df_elec_balans.index[0])
df_elec_balans = df_elec_balans.loc[df_elec_balans.index >= 1990]
df_hernieuwbare_productie['Netto elektriciteitsproductie percentage'] = df_hernieuwbare_productie['Netto elektriciteitsproductie percentage'].apply(lambda x: float(x) if type(x) == int else float(x.replace(',', '.')))
df_hernieuwbare_verbruik['Bruto eindverbruik relatief'] = df_hernieuwbare_verbruik['Bruto eindverbruik relatief'].apply(lambda x: float(x) if type(x) == int or type(x) == float else float(x.replace(',', '.')))
df_hernieuwbare_verbruik['Vermeden emissie relatief'] = df_hernieuwbare_verbruik['Vermeden emissie relatief'].apply(lambda x: float(x) if type(x) == int else float(x.replace(',', '.')))

In [ ]:
df_table_wind_turbine = pd.DataFrame()
df_table_wind_turbine['Windturbine (%)'] =  round(df_elec_prod_totaal['Windturbine Mwh'] / df_elec_prod_totaal['Totaal Mwh'] * 100,2)
df_table_wind_turbine.tail()

In [ ]:
ax = df_elec_prod_totaal.drop(['Totaal Mwh', 'Overige installaties  Mwh'], 1).plot(kind='bar', stacked=True)
ax.set_title('Elektriciteit productie Nederland per categorie', size=20)
ax.set_ylabel('Productie (MWh)', size=16)
ax.set_xlabel('Jaar', size=16)
ax.legend(loc='upper left')
ax.get_figure().savefig('output/productietotaal.svg', bbox_inches = 'tight')

In [ ]:
ax = df_elec_prod_totaal.loc[[2017]].drop(['Totaal Mwh', 'Overige installaties  Mwh'], 1).T.plot.pie(y=2017,autopct='%1.1f%%',
        startangle=90, pctdistance=0.85)
ax.legend().remove()
ax.set_ylabel('')
ax.axis('equal')
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
ax.set_title('Energie Productie per categorie 2017', size=20, y=1.02)
ax.get_figure().savefig('output/productietotaal2017.svg', bbox_inches = 'tight')

In [ ]:
df1 = df_elec_balans.iloc[:, 0:6].copy()
df1['Balans'] = df1['Uitvoer'] - df1['Invoer']
df1['Decentraal (%)'] = df1['||Decentraal']/df1['||Totaal productie']*100
df1['Decentraal (%)'] = df1['Decentraal (%)'].apply(lambda x: round(x, 2))

In [ ]:
ax = df1['Balans'].plot.bar()
ax.set_title('Eletricititeits balans Nederland (uitvoer - invoer)', size=20)
ax.set_xlabel('Jaar', size=16)
ax.set_ylabel('Balans (ml kWh)', size=16)
ax.get_figure().savefig('output/balans.svg', bbox_inches = 'tight')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)
fig.suptitle('Decentrale Elektriciteit productie Nederland', size=20);
df1[['||Centraal', '||Decentraal']].plot.bar(stacked=True, ax=axes[0])
axes[0].set_ylabel('Productie (mln kWh)', size=16)
axes[0].set_xlabel('jaar', size=16)
df1['Decentraal (%)'].plot.line(ax=axes[1], grid=True, marker='*')
axes[1].set_ylim(15,40)
axes[1].set_xlim(1989, 2020)
axes[1].set_ylabel('Percentage (%)', size=16)
axes[1].set_xlabel('Jaar', size=16)

fig.savefig('output/decentraleproductie.svg', bbox_inches = 'tight')

Binnenlandse productie van hernieuwbare elektriciteit als percentage van het totale binnenlandse elektriciteitsverbruik. De hernieuwbare elektriciteitsproductie wordt afgezet tegen het totale elektriciteitsverbruik en niet tegen de totale elektriciteitsproductie. Deze keuze vloeit voort uit Europese afspraken.

In [ ]:
df2 = df_hernieuwbare_productie[['Netto elektriciteitsproductie', 'Bron/Techniek mln kWh', 'Perioden']]
grouped = df2.groupby('Perioden')
dfs = []
for jaar, df in grouped:
    df = df.set_index('Bron/Techniek mln kWh').drop('Perioden', 1).T[['Waterkracht', 'Totaal windenergie', 'Zonnestroom', 'Totaal biomassa', 'Totaal biogas']]
    df.index = [jaar]
    df.index.name = 'Perioden'
    dfs.append(df)
df2 = pd.concat(dfs)

df3 = df_hernieuwbare_productie[df_hernieuwbare_productie['Bron/Techniek mln kWh'] == 'Totaal hernieuwbare energiebronnen']
df3 = df3.set_index('Perioden').drop('Bron/Techniek mln kWh', 1)

In [ ]:
df3['Netto elektriciteitsproductie percentage'].head()

In [ ]:
df3['Netto elektriciteitsproductie percentage'].tail()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)
fig.suptitle('Energie Productie Uit Herniewbare Bronnen', size=20)

df2.plot.bar(stacked=True, ax=axes[0])
axes[0].set_xlabel('Jaar', size=16)
axes[0].set_ylabel('Productie (mln kWh)', size=16)
axes[0].set_title('Productie Absoluut per Categorie', size=18)

df3['Netto elektriciteitsproductie percentage'].plot.line(marker='*', grid=True, ax=axes[1])
axes[1].set_xlabel('Jaar', size=16)
axes[1].set_ylabel('Productie (%)',size=16)
axes[1].set_title('Productie Percentage van Verbruik', size=18)
axes[1].set_xlim(1989, 2020)
axes[1].set_ylim(0, 15)

fig.savefig('output/hernieuwbareproductie.svg', bbox_inches = 'tight')

In [ ]:
df4 = df_hernieuwbare_verbruik[df_hernieuwbare_verbruik['onderwerp'] == 'Totaal energiebronnen'].drop('onderwerp',1)
df4 = df4[df4['Energietoepassingen'] != 'Totaal energietoepassingen']
df4 = df4[['Perioden', 'Bruto eindverbruik (TJ)', 'Energietoepassingen']]
# df4 = df4[df4['Energietoepassingen'] == 'Bruto eindverbruik (TJ)']
grouped = df4.groupby('Perioden')
jaar, df = list(grouped)[0]
dfs = []
for jaar, df in grouped:
    df = df.drop('Perioden', 1).set_index('Energietoepassingen').T
    df.index = [int(jaar)]
    df.index.name = 'Perioden'
    dfs.append(df)
df4 = pd.concat(dfs)

df5 = df_hernieuwbare_verbruik[(df_hernieuwbare_verbruik['onderwerp'] == 'Totaal energiebronnen')&(df_hernieuwbare_verbruik['Energietoepassingen'] == 'Totaal energietoepassingen')]
df5 = df5[['Perioden', 'Bruto eindverbruik relatief']].set_index('Perioden')

In [ ]:
df5.tail()

In [ ]:
df5.tail()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)
fig.suptitle('Energie Verbruik Uit Hernieuwbare Bronnen', size=20)
df4.plot.bar(stacked=True, ax=axes[0])
df5.plot.line(ax=axes[1], marker='*', grid=True)

axes[0].set_title('Verbruik Absoluut per Categorie', size=18)
axes[0].set_xlabel('Jaar', size=16)
axes[0].set_ylabel('Verbruik (TJ)')

axes[1].set_title('Verbruik Percentage van Totale Verbruik', size=18)
axes[1].legend().remove()
axes[1].set_ylim(0, 8)
axes[1].set_xlim(1989, 2020)
axes[1].set_xlabel('Jaar', size=16)
axes[1].set_ylabel('Verbruik (%)')

fig.savefig('output/hernieuwbareverbruik.svg', bbox_inches = 'tight')